In [1]:
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mne
from bids import BIDSLayout
from src.pipelines.p100_pipeline import P100AnalysisPipeline
from config import IMAGES_DIR  # or replace with your path directly


def plot_grand_average_p100_custom(bids_dir, visual_config, rest_config, channels=['PO3', 'POz', 'PO4']):
    layout = BIDSLayout(bids_dir, validate=True)
    subject_ids = layout.get_subjects()

    all_evokeds_1 = []
    all_evokeds_2 = []

    for sub in subject_ids:
        session_ids = layout.get_sessions(subject=sub) or ['01']
        for ses in session_ids:
            try:
                pipeline = P100AnalysisPipeline(
                    subject_id=sub,
                    session_id=ses,
                    condition1_config=visual_config,
                    condition2_config=rest_config,
                    channels=channels
                ).load_data().build_epochs()

                evoked_1 = pipeline.epochs1.copy().pick_channels(channels).average()
                evoked_2 = pipeline.epochs2.copy().pick_channels(channels).average()

                all_evokeds_1.append(evoked_1)
                all_evokeds_2.append(evoked_2)

            except Exception as e:
                print(f"Skipping sub-{sub}, ses-{ses}: {e}")

    if not all_evokeds_1 or not all_evokeds_2:
        print("No valid evoked data found.")
        return

    grand_avg_1 = mne.grand_average(all_evokeds_1)
    grand_avg_2 = mne.grand_average(all_evokeds_2)

    # Pick and extract data
    try:
        ch_idx_1 = [grand_avg_1.ch_names.index(ch) for ch in channels]
        ch_idx_2 = [grand_avg_2.ch_names.index(ch) for ch in channels]
    except ValueError as e:
        raise ValueError(f"Channel not found in grand average: {e}")

    data_1 = grand_avg_1.data[ch_idx_1, :] * 1e6  # µV
    data_2 = grand_avg_2.data[ch_idx_2, :] * 1e6  # µV

    min_len = min(data_1.shape[1], data_2.shape[1])
    times = grand_avg_1.times[:min_len]

    data_1 = data_1[:, :min_len]
    data_2 = data_2[:, :min_len]

    mean_1 = data_1.mean(axis=0)
    mean_2 = data_2.mean(axis=0)

    ch_str_1 = ', '.join(channels)
    ch_str_2 = ', '.join(channels)

    # Plotting
    # Set Seaborn style
    sns.set_style("whitegrid")

    # Create the figure
    plt.figure(figsize=(12, 6))  # Slightly wider for clarity

    # Highlight time window
    plt.axvspan(0.080, 0.120, color='orange', alpha=0.3, label='80–120 ms window')
    plt.axvline(0, color='black', linestyle='--', linewidth=1.5, label='Stimulus Onset')

    # Plot the signals
    plt.plot(times, mean_1, label=f'{visual_config["label"]} (mean) [{ch_str_1}]',
            linestyle='-', linewidth=2.5, color='#1f77b4')
    plt.plot(times, mean_2, label=f'{rest_config["label"]} (mean) [{ch_str_2}]',
            linestyle='--', linewidth=2.5, color='#ff7f0e')

    # Axis labels with larger font
    plt.xlabel('Time (s)', fontsize=18, fontweight='bold')
    plt.ylabel('Amplitude (µV)', fontsize=18, fontweight='bold')

    # Legend styling
    plt.legend(loc='best', fontsize=11, frameon=True)

    # Ticks styling
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)

    # Grid styling
    plt.grid(True, which='major', linestyle='--', linewidth=0.5, alpha=0.6)

    # Remove top and right spines
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # Tight layout for clean spacing
    plt.tight_layout()

    # Save the figure
    out_dir = Path(IMAGES_DIR, "grand_average")
    out_dir.mkdir(parents=True, exist_ok=True)
    out_path = out_dir / f'final/p100_grand_average_{visual_config["label"]}_{rest_config["label"]}.png'
    plt.savefig(out_path, dpi=600)
    plt.close()


In [2]:
import config as config
if config.P_100_ANALYSIS:
        visual = {
            "label": "Visual Change",
            "trial_type": "Stimulus",
            "tmin": -0.2,
            "tmax": 0.5,
            "trial_mode": "",
            "trial_unit": "Words",
            "experiment_mode": "Experiment",
            "trial_boundary": "Start",
            "modality": "Pictures"
        }

        rest = {
            "label": "No Visual Change",
            "trial_type": "Fixation",
            "tmin": -0.2,
            "tmax": 0.5,
            "trial_mode": "",
            "trial_unit": "Words",
            "experiment_mode": "Experiment",
            "trial_boundary": "Start",
            "modality": "Pictures",
            "time_window": (0.08, 0.12)  # Optional window for P100
        }

        cond1, cond2 = plot_grand_average_p100_custom(config.BIDS_DIR, visual_config=visual, rest_config=rest)

╭─────────────────────────────────────────╮
│ 🚀 Initializing BIDSDatasetReader Class │
╰─────────────────────────────────────────╯

Loading Processed EEG Data: sub-01 ses-01

╭─────────────────────────────────────╮
│  Initializing EEGEpochBuilder Class │
╰─────────────────────────────────────╯

Creating EPOCHS

        Extraction Criteria        
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Parameter          ┃ Value      ┃
┣━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━┫
┃ 🧪 Trial Mode      ┃            ┃
┃ 📏 Trial Unit      ┃ Words      ┃
┃ 🔗 Experiment Mode ┃ Experiment ┃
┃ 🎭 Trial Boundary  ┃ Start      ┃
┃ 🔬 Trial Type      ┃ Stimulus   ┃
┃ 📊 Modality        ┃ Pictures   ┃
┃ ⏱ Tmin             ┃ -0.2       ┃
┃ ⏳ Tmax            ┃ 0.5        ┃
┗━━━━━━━━━━━━━━━━━━━━┻━━━━━━━━━━━━┛

...

╭─────────────────────────────────────╮
│  Initializing EEGEpochBuilder Class │
╰─────────────────────────────────────╯

Creating EPOCHS

        Extraction Criteria        
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Parameter          ┃ Value      ┃
┣━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━┫
┃ 🧪 Trial Mode      ┃            ┃
┃ 📏 Trial Unit      ┃ Words      ┃
┃ 🔗 Experiment Mode ┃ Experiment ┃
┃ 🎭 Trial Boundary  ┃ Start      ┃
┃ 🔬 Trial Type      ┃ Fixation   ┃
┃ 📊 Modality        ┃ Pictures   ┃
┃ ⏱ Tmin             ┃ -0.2       ┃
┃ ⏳ Tmax            ┃ 0.5        ┃
┗━━━━━━━━━━━━━━━━━━━━┻━━━━━━━━━━━━┛

Not setting metadata
61 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 61 events and 701 original time points ...
0 bad epochs dropped
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Identifying common channels ...
Identifying common channels ...


TypeError: cannot unpack non-iterable NoneType object